In [65]:
# !/usr/bin/python
# -*- coding: utf-8 -*-
# Author: Selvaria
# 筛选企业分属生产厂家/经销商还是其他关系不大的企业

import pandas

class StationFilter(object):
    
    def __init__(self, in_dir, out_dir1, out_dir2=None):
        '''
        对象需传入：
        in_dir:原始表格地址；
        out_dir1:需求的分类厂家输出表格地址；
        out_dir2:其他企业的输出表格地址。
        '''
        self.in_dir = in_dir
        self.out_dir1 = out_dir1
        self.out_dir2 = out_dir2
        
    def get_dataframe(self, sheet_name=0):
        #df_province.to_excel(self.in_dir, index=False)
        df_province = pandas.read_excel(self.in_dir, sheet_name=sheet_name)
        return df_province
    
    def filter_name(self): #删掉无用的行，注意调整表头名称和关键字
        df_province = self.get_dataframe()
        del_list = []
        for i in range(len(df_province)):
            name = df_province.loc[i, '企业名称']
            phone = df_province.loc[i, '电话号码']
            #print(phone)
            if '中国石化' in name or '中国石油' in name or '中海' in name:
                del_list.append(i)
            #if '加油' not in name or '加气' not in name:
                #del_list.append(i)
            if '-' in phone:
                del_list.append(i)
        df_new = df_province.drop(df_province.index[del_list],axis=0,inplace=False) 
        return df_new
    
    def filter_name2(self): #删掉无用的行，注意调整表头名称和关键字
        df_province = self.get_dataframe()
        del_list = []
        for i in range(len(df_province)):
            name = df_province.loc[i, '企业名称']
            phone = df_province.loc[i, '电话号码']
            #print(phone)
            if '加油站' in name or '加气' in name:
                del_list.append(i)
            #if '加油' not in name or '加气' not in name:
                #del_list.append(i)
        df_new = df_province.drop(df_province.index[del_list],axis=0,inplace=False) 
        return df_new
    
    def select_oil_producer(self, df): #精确提取原油生产厂家
        df_province = df
        select_list = []
        for index in df_province.index:
            name = df_province.loc[index, '企业名称']
            scope = df_province.loc[index, '经营范围']
            if '炼' in name or '烷' in scope or '苯' in scope or '醚' in scope or '醇' in scope or '油' in scope:
                select_list.append(index)
        df_new = df_province.loc[select_list]
        return df_new
    
    def select_name(self, mode=0): #选取需要的行，删掉无用的行，并分别返回dataframe
        '''
        mode:
        0：直接返回准确油品炼厂（用于筛选生产企业）
        1：返回相对概略的相关厂家（一般用于筛选经销商）
        '''
        df_province = self.get_dataframe()
        select_list = []
        #del_list = []
        for i in range(len(df_province)):
            name = str(df_province.loc[i, '企业名称'])
            scope = str(df_province.loc[i, '经营范围'])
            #if type(scope) == float:
                #print('经营范围数据格式错误',i)
                #scope = str(scope)
                #continue
            if '科技' in name or '设备' in name or '机械' in name or '研究' in name or '铸造' in name or '仪' in name or '轴承' in name:
                select_list.append(i)
            if '纺织' in name or '建材' in name or '盐' in name or '家具' in name or '办公' in name or '矿' in name or '工艺' in name:
                select_list.append(i)
            if '馍' in name or '日化' in name or '食品' in name or '鞋' in name or '日用' in name or '玻璃' in name or '瓷' in name:
                select_list.append(i)
            if '加油' in name or '加气' in name or '研' in name or '信息' in name:
                select_list.append(i)
            if '设备' in scope or '仪' in scope or '配件' in scope or '机械' in scope or '纸' in scope or '妆' in scope:
                select_list.append(i)
            if '茶' in scope or '大理石' in scope or '标准件' in scope or '服装' in scope or '鞋' in scope or '报刊' in scope:
                select_list.append(i)
            if '百货' in scope or '菜' in scope or '书' in scope or '食' in scope or '农' in scope or '矿' in scope:
                select_list.append(i)
            if '肉' in scope or '工艺品' in scope or '皮革' in scope or '漂' in scope or '五金' in scope or '建筑' in scope:
                select_list.append(i)
            if '木' in scope or '家具' in scope or '布' in scope or '管件' in scope or '钢' in scope or '紧固件' in scope:
                select_list.append(i)
            if '香油' in scope or '磨油' in scope or '管道' in scope or '钛' in scope or '铝' in scope:
                select_list.append(i)
        select_unique = []
        for index in select_list:
            if index not in select_unique:
                select_unique.append(index)
        df_need_middle = df_province.drop(df_province.index[select_unique],axis=0,inplace=False) 
        if mode == 0:
            df_need = self.select_oil_producer(df_need_middle)
        else:
            df_need = df_need_middle
        df_drop = df_province.loc[select_unique]
        return df_need, df_drop
    
    def output_result(self, mode=0):
        #df_new = self.filter_name()
        #df_drop = self.filter_name2()
        if mode == 0:
            df_need, df_drop = self.select_name()
        else:
            df_need, df_drop = self.select_name(mode=1)
        df_drop.to_excel(self.out_dir2, index=False)
        df_need.to_excel(self.out_dir1, index=False)
        print('筛选完成')
        

        
def test():
    in_dir = r'D:\data\qichacha\洛阳市\20190327131981257927008412_16734898石化制造.xls'
    out_dir = r'D:\data\qichacha\洛阳市\洛阳石化生产企业.xls'
    out_dir2 = r'D:\data\qichacha\洛阳市\洛阳石化相关企业.xls'
    sf = StationFilter(in_dir, out_dir, out_dir2)
    df_n, df_d = sf.select_name()
    return df_n
    #return df_d
    
def oil_producer():
    in_dir = r'D:\data\qichacha\洛阳市\洛阳周边.xlsx'
    out_dir = r'D:\data\qichacha\洛阳市\洛阳炼厂.xls'
    sf = StationFilter(in_dir, out_dir)
    df = sf.select_oil_producer()
    df.to_excel(out_dir, index=False)
    print('筛选完成')
    
def main():
    #in_dir = r'D:\data\qichacha\河南全省\石化企业河南生产厂家.xls'
    in_dir = r'D:\data\qichacha\河南全省\石化企业河南经销商.xls'
    #out_dir = r'D:\data\qichacha\河南全省\河南生产石化企业筛选后.xls'
    out_dir = r'D:\data\qichacha\河南全省\河南经销商石化企业筛选后.xls'
    out_dir2 = r'D:\data\qichacha\河南全省\河南其他石化相关企业筛选后.xls'
    sf = StationFilter(in_dir, out_dir, out_dir2)
    #sf.output_result()
    sf.output_result(mode=1)
    
if __name__ == "__main__":
    #df = test()
    main()
    #oil_producer()

筛选完成


In [20]:
df

,企业名称,省份,城市,统一社会信用代码,法定代表人,企业类型,成立日期,注册资本,地址,邮箱,经营范围,网址,电话号码,电话号码（更多号码）
0,洛阳石化吉润化工有限责任公司,河南,洛阳市,914103061710843331,赵振辉,其他有限责任公司,1998-04-21,14000万元人民币,洛阳市吉利区大港路,2537722586@qq.com,生产:氢气、氮[压缩的]、聚丙烯[中间产品氢气属于危险化学品](安全生产许可证(豫O)WH安...,NaN,0379-66996605,13783111015；0379-66997392；
7,洛阳市德泉石化有限公司,河南,洛阳市,914103225828547343,于宁宁,有限责任公司(自然人投资或控股),2011-09-14,7000万元人民币,孟津县华阳产业集聚区(白鹤镇),149546248@qq.com,一般经营项目:炼油助剂、水处理药剂(危险化学品除外)生产、销售。,www.lydqsh.com,0379-63911011,0379-67862272；
18,洛阳德能石化有限公司,河南,洛阳市,914103065710260794,董贯通,有限责任公司(自然人独资),2011-03-15,2000万元人民币,吉利区华北路中段,873820844@qq.com,生产销售:炼油助剂、水处理剂(不含有毒有害易燃易爆危险品)。,NaN,17737730752,13783193025；0379-67862272；0379-67862556；
22,洛阳市建发石化有限公司,河南,洛阳市,91410381706542741A,张留见,有限责任公司(自然人投资或控股),2004-04-01,1800万元人民币,偃师市顾县镇顾县村,yttysj@163.com,群青、净水剂(高效絮凝破乳剂)的生产销售。,NaN,0379-67517707,NaN
52,河南省开通石化管道有限公司,河南,洛阳市,91410381099533009C,曲中厚,有限责任公司(自然人投资或控股),2014-05-09,500万元人民币,偃师市顾县镇曲家寨村,1106682566@qq.com,钢制高、中、低压管件、法兰、锻件的生产、销售;钢材的销售。,NaN,13663796094,0379-67518039；
54,洛阳高新恒力石化装备有限公司,河南,洛阳市,914103007112911745,郭子明,有限责任公司（非自然人投资或控股的法人独资）,1998-12-17,500万元人民币,中国(河南)自由贸易试验区洛阳片区高新开发区凌波路西路,cddcdd2007@163.com,石油化工试剂、催化剂研发、生产、销售。,NaN,0379-61298978,0379-61298979；0379-63061787；0379-69766897；
60,洛阳信众石化有限公司,河南,洛阳市,914103007338596058,张明辉,有限责任公司(自然人独资),2001-11-28,200万元人民币,洛阳高新开发区金达路怡景园8-1-406室,luoyangxinzhong@163.com,轧制乳化液合成酯的生产、销售。,NaN,0379-64111518,15713795598；
65,河南省源通石化机械有限公司,河南,洛阳市,9141038158287740X0,曲晨辉,有限责任公司(自然人投资或控股),2011-09-22,180万元人民币,偃师市顾县镇曲家寨工业区,276860659@qq.com,钢制管件、法兰、金属垫片的生产销售。,NaN,18937932222,0379-67521388；0379-67773899；
67,洛阳金达石化有限责任公司化工厂,河南,洛阳市,NaN,王亚丁,有限责任公司分公司(自然人投资或控股),2007-05-16,NaN,吉利区华北路中段,lyjdzy@126.com,生产销售:水处理剂、净水剂、水质稳定剂、破乳剂、炼油助剂、杀菌剂(涉及许可经营的凭有效许可证...,NaN,0379-66992006,0379-66991346；13683840689；0379-66991922；
69,洛阳金达石化有限责任公司印刷厂,河南,洛阳市,NaN,郭夏明,有限责任公司分公司(自然人投资或控股),2007-07-03,NaN,洛阳市吉利区胜利路北段,lyjdzy@126.com,出版物印刷。,NaN,0379-66992006,NaN


In [64]:
import xlrd  
import xlwt  
import datetime
import glob

now = datetime.datetime.now()
today = now.strftime('%Y-%m-%d')

def merge_file(origin_dir, new_dir, name=today):
    #date = today
    biaotou=['企业名称','省份','城市','统一社会信用代码','法人','企业类型','成立日期','注册资本','地址','邮箱','经营范围','网址','电话号码','其他号码']  

    filelocation = origin_dir  #搜索多个表格存放处 
    fileform="xls"  #当前文件夹下搜索的文件名后缀  

    filedestination = new_dir #将合并后的表格存放到的位置  
    file="石化企业{}".format(name) #合并后的表格名 

    #首先查找默认文件夹下有多少文档需要整合  
     
    filearray=[]  
    for filename in glob.glob(filelocation+"*."+fileform):  
        filearray.append(filename)  
    #以上是从filelocation文件夹下读取所有excel表格，并将所有的名字存储到列表filearray 

    print("在默认文件夹下有%d个文档"%len(filearray))  
    ge=len(filearray)  
    matrix = [None]*ge  
    #实现读写数据  

    #下面是将所有文件读数据到三维列表cell[][][]中（不包含表头）  
    for i in range(ge):  
        fname=filearray[i]  
        bk=xlrd.open_workbook(fname)  
        try:  
            #sh=bk.sheet_by_name("1")  #下载的sheet名字
            #sh=bk.sheet_by_name(fname[-39:-8]) 
            sh=bk.sheet_by_name(fname[-44:-13]) 
        except:  
            print ("在文件%s中没有找到sheet1，读取文件数据失败,注意表格sheet的名字" %fname)  
        nrows=sh.nrows   
        matrix[i] = [0]*(nrows-1)  

        ncols=sh.ncols  
        for m in range(nrows-1):    
            matrix[i][m] = ["0"]*ncols  

        for j in range(1,nrows):  
            for k in range(0,ncols):  
                matrix[i][j-1][k]=sh.cell(j,k).value  

    #下面是写数据到新的合并表格中  
    filename=xlwt.Workbook()  
    sheet=filename.add_sheet(name)  #新表格的sheet名（name）

    #下面是把表头写上  
    for i in range(0,len(biaotou)):  
        sheet.write(0,i,biaotou[i])  

    #求和前面的文件一共写了多少行  
    zh=1  
    for i in range(ge):  
        for j in range(len(matrix[i])):  
            for k in range(len(matrix[i][j])):  
                sheet.write(zh,k,matrix[i][j][k])  
            zh=zh+1  
    print("我已经将%d个文件合并成1个文件，并命名为%s.xls."%(ge,file))  
    filename.save(filedestination+file+".xls")  
    

'''#origin_dir = r'D:\data\qichacha\洛阳市\生产\\'
origin_dir = r'D:\data\qichacha\洛阳市\销售\\'
new_dir = r'D:\data\qichacha\洛阳市\\'
#merge_file(origin_dir, new_dir, name='洛阳市生产')
merge_file(origin_dir, new_dir, name='洛阳市销售')'''

#origin_dir = r'D:\data\qichacha\河南全省\生产厂家\\'
origin_dir = r'D:\data\qichacha\河南全省\经销商\\'
new_dir = r'D:\data\qichacha\河南全省\\'
#merge_file(origin_dir, new_dir, name='河南生产厂家')
merge_file(origin_dir, new_dir, name='河南经销商')

在默认文件夹下有15个文档
我已经将15个文件合并成1个文件，并命名为石化企业河南经销商.xls.


In [73]:
dir_all = r'D:\data\qichacha\河南全省\河南全省.xlsx'
df_product = pandas.read_excel(dir_all, sheet_name=1)
df_trade = pandas.read_excel(dir_all, sheet_name=2)
df_rela = pandas.read_excel(dir_all, sheet_name=3)

def select_name(df): #选取需要的行，删掉无用的行，并分别返回dataframe

    #df_province = self.get_dataframe()
    df_province = df
    select_list = []
    #del_list = []
    for i in range(len(df_province)):
        name = str(df_province.loc[i, '企业名称'])
        scope = str(df_province.loc[i, '经营范围'])
        #if type(scope) == float:
            #print('经营范围数据格式错误',i)
            #scope = str(scope)
            #continue
        if '科技' in name or '设备' in name or '机械' in name or '研究' in name or '铸造' in name or '仪' in name or '轴承' in name:
            select_list.append(i)
        if '纺织' in name or '建材' in name or '盐' in name or '家具' in name or '办公' in name or '矿' in name or '工艺' in name:
            select_list.append(i)
        if '馍' in name or '日化' in name or '食品' in name or '鞋' in name or '日用' in name or '玻璃' in name or '瓷' in name:
            select_list.append(i)
        #if '加油' in name or '加气' in name or '研' in name or '信息' in name:
            #select_list.append(i)
        if '设备' in scope or '仪' in scope or '配件' in scope or '机械' in scope or '纸' in scope or '妆' in scope:
            select_list.append(i)
        if '茶' in scope or '大理石' in scope or '标准件' in scope or '服装' in scope or '鞋' in scope or '报刊' in scope:
            select_list.append(i)
        if '百货' in scope or '菜' in scope or '书' in scope or '食' in scope or '农' in scope or '矿' in scope:
            select_list.append(i)
        if '肉' in scope or '工艺品' in scope or '皮革' in scope or '漂' in scope or '五金' in scope or '建筑' in scope:
            select_list.append(i)
        if '木' in scope or '家具' in scope or '布' in scope or '管件' in scope or '钢' in scope or '紧固件' in scope:
            select_list.append(i)
        if '香油' in scope or '磨油' in scope or '管道' in scope or '钛' in scope or '铝' in scope:
            select_list.append(i)
    select_unique = []
    for index in select_list:
        if index not in select_unique:
            select_unique.append(index)
    df_need = df_province.drop(df_province.index[select_unique],axis=0,inplace=False) 
    #df_drop = df_province.loc[select_unique]
    return df_need

def select_addition_name(df): 

    df_province = df
    select_list = []
    #del_list = []
    for i in range(len(df_province)):
        name = str(df_province.loc[i, '企业名称'])
        scope = str(df_province.loc[i, '经营范围'])
        if '服务' in name or '油漆' in name or '实验' in name or '药' in name or '油墨' in name or '试验' in name or '粮' in name:
            select_list.append(i)
        #if '加油' in name or '加气' in name or '研' in name or '信息' in name:
            #select_list.append(i)
        if '肥' in scope or '汽车' in scope or '饲料' in scope or '粮' in scope or '装饰' in scope or '种子' in scope:
            select_list.append(i)
        if '陶粒' in scope or '耐火' in scope or '地址' in scope or '管件' in scope or '钛' in scope or '花' in scope:
            select_list.append(i)
        if '香油' in scope or '磨油' in scope or '管道' in scope or '油漆' in scope or '香蕉' in scope or '石英' in scope:
            select_list.append(i)
    select_unique = []
    for index in select_list:
        if index not in select_unique:
            select_unique.append(index)
    df_need = df_province.drop(df_province.index[select_unique],axis=0,inplace=False) 
    df_drop = df_province.loc[select_unique]
    return df_need, df_drop

#df_need = select_addition_name(df_rela)
#df_need, df_drop = select_addition_name(df_product)
df_need, df_drop = select_addition_name(df_trade)

test_dir =  r'D:\data\qichacha\河南全省\筛选后企业.xls'

writer = pandas.ExcelWriter(test_dir)
df_need.to_excel(writer, sheet_name= 'need', index=False)
df_drop.to_excel(writer, sheet_name= 'drop', index=False)
writer.save()
print('筛选完成')

筛选完成
